# Linear Regression Code Along

This notebook is the reference for the video lecture on the Linear Regression Code Along. Basically what we do here is examine a dataset with Ecommerce Customer Data for a company's website and mobile app. Then we want to see if we can build a regression model that will predict the customer's yearly spend on the company's product.

First thing to do is start a Spark Session

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
# Use Spark to read in the Ecommerce Customers csv file.
data = spark.read.csv("Ecommerce_Customers.csv",inferSchema=True,header=True)

In [4]:
# Print the Schema of the DataFrame
data.printSchema()

# Avatar: 색상을 의미
# Avg Session Length: 평균 세션 시간
# Time on App: 앱 사용 시간
# Time on Website:  웹사이트 사용 시간
# Length of Membership: 멤버쉽 기간
# Yearly Amount Spent: 연간 지출액

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [6]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [7]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


## Setting Up DataFrame for Machine Learning 

In [8]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
# feature들을 하나의 벡터로 만들기위해 사용한다.
# 하나의 벡터로 만드는 이유는 알고리즘에 사용하는 feature 데이터 형식을 맞추기 위함
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [9]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [10]:
# inputCols: 벡터로 만들어줄 컬럼이름들(수치형 데이터만을 사용했음)
# ouputCols: 묶어서 나타낼 컬럼이름(반드시 'features'일 필요없다.)
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [11]:
output = assembler.transform(data)

In [12]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [13]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [14]:
output.head(1)
# features 컬럼이 생성된것을 볼 수 있다.
# transform에 사용된 컬럼도 그대로이다.

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [15]:
final_data = output.select("features",'Yearly Amount Spent')

In [17]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [18]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [19]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                346|
|   mean|  501.4403794414049|
| stddev|  80.88306729761456|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [20]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                154|
|   mean|  494.5366743023978|
| stddev|  75.71156783713344|
|    min| 256.67058229005585|
|    max|  689.2356997616951|
+-------+-------------------+



In [21]:
# Create a Linear Regression Model object
# featuresCol='features', labelCol='label', predictionCol='prediction'
# labelCol은 기본값과 다르므로 바꾼다.
# featuresCol에서 'features'이 기본값이므로 feature들을 벡터로 만들 때
# outputCol을 'features'로 지정하는것이 편해보인다.
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [22]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data,)

In [25]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.34652553888964,38.979312273686595,0.10160574244722508,61.32805350867041] Intercept: -1028.6015138325777


In [26]:
test_results = lrModel.evaluate(test_data)

In [27]:
# Interesting results....
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  -7.04249269667099|
|  5.809568952637164|
|-22.778901577179454|
|-2.3524172091102855|
| -5.389680538618222|
| 1.7842056556329453|
| 0.3530208243173547|
| -4.916577945154586|
| -5.455914245135318|
|-18.772493408044966|
|-15.148564932426325|
|-1.6962525075683743|
|-27.082429485217858|
|  -6.76051680007879|
| 1.9662445494214467|
|-19.192210364518417|
| 7.1140331578581595|
|-11.732632809808251|
| -9.494556918198839|
| 11.380467956848236|
+-------------------+
only showing top 20 rows



In [28]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 9.472843343190716
MSE: 89.73476100463265


In [29]:
final_data.describe().show()
# 'Yearly Amount Spent'의 표준편차 79.3148에 비해
# RMSE 9.4728은 좋은 수치이다.

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [30]:
unlabeled_data = test_data.select('features')

In [31]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.4925366965402...|
|[30.9716756438877...|
|[31.1239743499119...|
|[31.2606468698795...|
|[31.2681042107507...|
|[31.3091926408918...|
|[31.3895854806643...|
|[31.4252268808548...|
|[31.5147378578019...|
|[31.5702008293202...|
|[31.5741380228732...|
|[31.6610498227460...|
|[31.6739155032749...|
|[31.7207699002873...|
|[31.7366356860502...|
|[31.8164283341993...|
|[31.8512531286083...|
|[31.8648325480987...|
|[31.8854062999117...|
|[31.9262720263601...|
+--------------------+
only showing top 20 rows



In [32]:
predictions = lrModel.transform(unlabeled_data)

In [33]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...| 289.5137384165855|
|[30.9716756438877...|488.82904080425556|
|[31.1239743499119...| 509.7259554169452|
|[31.2606468698795...|423.67904846606166|
|[31.2681042107507...|428.86021371244215|
|[31.3091926408918...| 430.9365121843007|
|[31.3895854806643...|409.71659023566554|
|[31.4252268808548...| 535.6832965999165|
|[31.5147378578019...|495.26840224159673|
|[31.5702008293202...| 564.7179855494498|
|[31.5741380228732...| 559.5578370930132|
|[31.6610498227460...| 418.0546060874692|
|[31.6739155032749...|502.80749739509906|
|[31.7207699002873...| 545.5354502781017|
|[31.7366356860502...| 494.9672017061105|
|[31.8164283341993...| 520.3147018681748|
|[31.8512531286083...|465.87821350894023|
|[31.8648325480987...| 451.6239132866219|
|[31.8854062999117...|399.59782989067435|
|[31.9262720263601...|380.82446548747816|
+--------------------+------------

Excellent results! Let's see how you handle some more realistically modeled data in the Consulting Project!